In [66]:
import pandas as pd

In [67]:
# lire le dataset
data = pd.read_csv('Car_details_v3.csv')

In [68]:
data.head()

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Individual,Manual,First Owner,23.4 kmpl,1248 CC,74 bhp,190Nm@ 2000rpm,5.0
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Individual,Manual,Second Owner,21.14 kmpl,1498 CC,103.52 bhp,250Nm@ 1500-2500rpm,5.0
2,Honda City 2017-2020 EXi,2006,158000,140000,Petrol,Individual,Manual,Third Owner,17.7 kmpl,1497 CC,78 bhp,"12.7@ 2,700(kgm@ rpm)",5.0
3,Hyundai i20 Sportz Diesel,2010,225000,127000,Diesel,Individual,Manual,First Owner,23.0 kmpl,1396 CC,90 bhp,22.4 kgm at 1750-2750rpm,5.0
4,Maruti Swift VXI BSIII,2007,130000,120000,Petrol,Individual,Manual,First Owner,16.1 kmpl,1298 CC,88.2 bhp,"11.5@ 4,500(kgm@ rpm)",5.0


# acces à la colonne "name"

In [69]:
# Access a specific column
column_data = data['name']

# Display basic statistics of the column
print(column_data.describe())

count                       8128
unique                      2058
top       Maruti Swift Dzire VDI
freq                         129
Name: name, dtype: object


# enlever torque et owner

In [70]:
data = data.drop(columns=['torque','owner'])
data.head()

,name,year,selling_price,km_driven,fuel,seller_type,transmission,mileage,engine,max_power,seats
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Individual,Manual,23.4 kmpl,1248 CC,74 bhp,5.0
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Individual,Manual,21.14 kmpl,1498 CC,103.52 bhp,5.0
2,Honda City 2017-2020 EXi,2006,158000,140000,Petrol,Individual,Manual,17.7 kmpl,1497 CC,78 bhp,5.0
3,Hyundai i20 Sportz Diesel,2010,225000,127000,Diesel,Individual,Manual,23.0 kmpl,1396 CC,90 bhp,5.0
4,Maruti Swift VXI BSIII,2007,130000,120000,Petrol,Individual,Manual,16.1 kmpl,1298 CC,88.2 bhp,5.0


# enlever les lignes ayant une ou plusieurs cellules vides

In [71]:
data_cleaned = data.dropna(subset=['engine'],how='any',axis=0)
data.info( )


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8128 entries, 0 to 8127
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   name           8128 non-null   object 
 1   year           8128 non-null   int64  
 2   selling_price  8128 non-null   int64  
 3   km_driven      8128 non-null   int64  
 4   fuel           8128 non-null   object 
 5   seller_type    8128 non-null   object 
 6   transmission   8128 non-null   object 
 7   mileage        7907 non-null   object 
 8   engine         7907 non-null   object 
 9   max_power      7913 non-null   object 
 10  seats          7907 non-null   float64
dtypes: float64(1), int64(3), object(7)
memory usage: 698.6+ KB


## densité des differents fuels

In [72]:
densities = {
    "CNG": 0.656,
    "LPG": 0.55,
}
data_fragment = data_cleaned[["fuel", "mileage"]]
for index, row in data_fragment.iterrows():
    fuel_value = row['fuel']
    mileage_value = row['mileage']
   
    if 'km/kg' in mileage_value: 
        mileage_value = mileage_value.replace('km/kg', '')
        data_fragment.at[index, 'fuel'] = fuel_value
        mileage_value = float(mileage_value.strip()) / densities[fuel_value]
        data_fragment.at[index, 'mileage'] =  str(int(mileage_value)) + " kmpl"

data_cleaned.update(data_fragment)

data_cleaned.head()


C:\Users\Marwa\AppData\Local\Temp\ipykernel_21536\924799605.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cleaned.update(data_fragment)


,name,year,selling_price,km_driven,fuel,seller_type,transmission,mileage,engine,max_power,seats
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Individual,Manual,23.4 kmpl,1248 CC,74 bhp,5.0
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Individual,Manual,21.14 kmpl,1498 CC,103.52 bhp,5.0
2,Honda City 2017-2020 EXi,2006,158000,140000,Petrol,Individual,Manual,17.7 kmpl,1497 CC,78 bhp,5.0
3,Hyundai i20 Sportz Diesel,2010,225000,127000,Diesel,Individual,Manual,23.0 kmpl,1396 CC,90 bhp,5.0
4,Maruti Swift VXI BSIII,2007,130000,120000,Petrol,Individual,Manual,16.1 kmpl,1298 CC,88.2 bhp,5.0


## enlever les différentes unités de chaque colonne

In [73]:
import re

# Define a function to extract numeric values
def extract_numeric_value(text):
    numeric_value = re.findall(r'\d+\.*\d*', text)
    if numeric_value:
        return float(numeric_value[0])
    else:
        return None

def remove_unit(column):
   data_cleaned[column] = data_cleaned [column].apply(extract_numeric_value)

remove_unit('engine')
remove_unit('max_power')
remove_unit('mileage')

data_cleaned.head()

C:\Users\Marwa\AppData\Local\Temp\ipykernel_21536\1556159455.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cleaned[column] = data_cleaned [column].apply(extract_numeric_value)


,name,year,selling_price,km_driven,fuel,seller_type,transmission,mileage,engine,max_power,seats
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Individual,Manual,23.40,1248.0,74.00,5.0
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Individual,Manual,21.14,1498.0,103.52,5.0
2,Honda City 2017-2020 EXi,2006,158000,140000,Petrol,Individual,Manual,17.70,1497.0,78.00,5.0
3,Hyundai i20 Sportz Diesel,2010,225000,127000,Diesel,Individual,Manual,23.00,1396.0,90.00,5.0
4,Maruti Swift VXI BSIII,2007,130000,120000,Petrol,Individual,Manual,16.10,1298.0,88.20,5.0


## convertir kmpl en Litres par 100km

In [75]:
data_fragment = data_cleaned[["mileage"]]

for index, row in data_fragment.iterrows():
    mileage_value = row
    if float(mileage_value):
        mileage_value = 100 / float(mileage_value)

    data_fragment.at[index, "mileage"] = round(float(mileage_value), 2)

data_cleaned.update(data_fragment)

data_cleaned.head()

with open('data_cleaned.csv', 'w') as f:
    data_cleaned.to_csv(f, index=False)


C:\Users\Marwa\AppData\Local\Temp\ipykernel_21536\2857051863.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_cleaned.update(data_fragment)
